In [1]:
import logging
logging.getLogger('tensorflow').setLevel(logging.ERROR)
import cv2
import numpy as np

import torch
from pytorch_grad_cam import GradCAMPlusPlus
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget

from src.utils.opt import Config
from src.model import MODEL_REGISTRY
from demo.extractor.frame_extractor import FrameExtractor
from demo.extractor.face_fafi_extractor import FaceFAFIExtractor
from demo.utils.gradcam import GradCAMCompatibleModel
from demo.detector.lightning_detector import TorchLightningDetector

class MobileNetDetector(TorchLightningDetector):
    def __init__(self,
                 name, 
                 cfg_path: str="configs/inference/double_head_mobilenet_fafi_hybrid.yml",
                 frame_width=1280,
                 frame_height=720,
                 thickness_percentage=10, 
                 blur_percentage=10,
                 *args,
                 **kwargs
                ):
        super().__init__(name,
                         cfg_path, 
                         frame_width, 
                         frame_height, 
                         thickness_percentage, 
                         blur_percentage, 
                         *args,
                         **kwargs
                         )

    def create_gradcam_model(self, cfg):
        grad_cam_model = GradCAMCompatibleModel(self.create_detector_model(cfg))
        cam = GradCAMPlusPlus(model=grad_cam_model,
                            target_layers=[
                                grad_cam_model.model.img_extractor.extractor.blocks[-1],
                                grad_cam_model.model.img_variant_extractor.extractor.blocks[-1]
                                ],
                            use_cuda=torch.cuda.is_available(),
                            )
        cam.batch_size = cfg['data_loader']['test']['args']['batch_size']
        return grad_cam_model, cam
    
    def get_grad_cam(self, batch):
        n = batch["imgs"].shape[0]
        targets = [ClassifierOutputTarget(1) for i in range(n)]
        return self.cam(input_tensor=torch.cat((batch["imgs"], batch["img_variants"]), dim=1),
                                            targets=targets,
                                            eigen_smooth=False,
                                            aug_smooth=False)
    

import argparse
from pytorch_grad_cam.utils.image import show_cam_on_image   
    

/home/dtle/miniconda3/envs/fsd/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-09-09 18:46:48.700740: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-09 18:46:48.877347: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-09 18:46:49.597504: W tensorflow/compiler/xla/stream_executor/platform/d

In [2]:

import os
features = ['msr', 'fafi']
detector = {}
detector["msr"] = MobileNetDetector("mobilenet", "configs/inference/double_head_mobilenet_{}_hybrid.yml".format("msr"))
detector["fafi"] = MobileNetDetector("mobilenet", "configs/inference/double_head_mobilenet_{}_hybrid.yml".format("fafi"))

In [28]:
import pandas as pd
df = pd.read_csv("dataset_v1/imgs/val.csv")
video_list = df["filepath"].apply(lambda x: os.path.splitext(os.path.basename(x))[0]).to_list()
video_list = [os.path.join("dataset_v1/videos","deepfaker" if x.startswith("video") else "roop", "fake", '_'.join(x.split("_")[:-1])+".mp4") for x in video_list]
video_list = list(set(video_list))
print(video_list)

['dataset_v1/videos/roop/fake/200_ntthau.mp4', 'dataset_v1/videos/roop/fake/067.mp4', 'dataset_v1/videos/roop/fake/201_ntthau.mp4', 'dataset_v1/videos/roop/fake/068_nqktuyen.mp4', 'dataset_v1/videos/roop/fake/066_nqktuyen.mp4', 'dataset_v1/videos/deepfaker/fake/video008_ttha.mp4', 'dataset_v1/videos/deepfaker/fake/video022.mp4', 'dataset_v1/videos/roop/fake/030.mp4', 'dataset_v1/videos/roop/fake/034_sedanh.mp4', 'dataset_v1/videos/roop/fake/034.mp4', 'dataset_v1/videos/roop/fake/067_nqktuyen.mp4', 'dataset_v1/videos/deepfaker/fake/video008.mp4', 'dataset_v1/videos/deepfaker/fake/video104_nqduy.mp4', 'dataset_v1/videos/roop/fake/201.mp4', 'dataset_v1/videos/deepfaker/fake/video104.mp4', 'dataset_v1/videos/roop/fake/074.mp4', 'dataset_v1/videos/deepfaker/fake/video105.mp4', 'dataset_v1/videos/deepfaker/fake/video102.mp4', 'dataset_v1/videos/deepfaker/fake/video103_nqduy.mp4', 'dataset_v1/videos/roop/fake/202.mp4', 'dataset_v1/videos/roop/fake/069.mp4', 'dataset_v1/videos/roop/fake/204.mp

In [31]:
import shutil
video_list = ["dataset_v1/videos/roop/fake/200_ntthau.mp4"]
for video in video_list:
    if not os.path.exists(video): continue
    video_basename = os.path.splitext(os.path.basename(video))[0]
    for feature in features:
        x = detector[feature].predict(open(video, "rb").read(), sampling=10000)

        try:
            image_path= x.iloc[0]["predict"][0]["face_path"]
            shutil.copyfile(image_path, f"imgs/detector_cam_mobilenet_{video_basename}_source{os.path.splitext(image_path)[1]}")
            grayscale_cam = x.iloc[0]["predict"][0]["grad_cam"]
            print(grayscale_cam.shape)
            rgb_img = cv2.imread(image_path, 1)[:, :, ::-1]
            rgb_img = cv2.resize(rgb_img, (224, 224))
            rgb_img = np.float32(rgb_img) / 255

            cam_image = show_cam_on_image(rgb_img, grayscale_cam)
            cv2.imwrite(f'imgs/detector_cam_mobilenet_{video_basename}_{feature}.jpg', cam_image)
        except Exception as e:
            continue

100%|██████████| 719/719 [00:00<00:00, 983.39it/s] 
/home/dtle/hnttruc-local/face-spoofing-dection-ABFTSCNN/demo/detector/lightning_detector.py:98: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  logits = F.softmax(logits)[:, 1]


(224, 224)


100%|██████████| 719/719 [00:00<00:00, 1029.88it/s]


(224, 224)
